Contains the base ideas for our linear program code

In [1]:
from pyscipopt import Model, quicksum
from simulator import combat_sim
from simulator import get_health
from simulator import get_damage
from sim_units import get_Units, get_Terran, get_Protoss, get_Zerg

In [2]:
from itertools import product

In [3]:
import json

This line here will eventually be swapped out with a function call to read our database of unit stats

In [4]:
Units = get_Units()
Terran_Units = get_Terran()
Protoss_Units = get_Protoss()
Zerg_Units = get_Zerg()

Viability function copied directly from earlier, separate snippet

In [ ]:
def viability(enemy_comp, test_comp, test_type='units'):
    """
    Input is the enemy army composition, the army composition that
    we are testing, and the type of viablity test we are running.
    test_type is a string, either "units", "health", or "damage"
    Runs simulation 200 times. If the test army has at
    least 10% of its test type remaining 90% of the time, return 1
    Otherwise return 0
    """
    test_limit = 0
    if test_type == 'units':
        unit_count = 0
        for unit in test_comp:
            unit_count += test_comp[unit]
        test_limit = 0.1 * unit_count
    
    if test_type == 'health':
        army_health = 0
        for unit in test_comp:
            unit_hp = Units[unit]['hp']
            army_health += unit_hp * test_comp[unit]
        test_limit = 0.1 * army_health
    if test_type == 'damage':
        army_dmg = 0
        for unit in test_comp:
            unit_dmg = Units[unit]['dps']
            army_dmg += unit_dmg * test_comp[unit]
        test_limit = 0.1 * army_dmg
    
    wins = 0
    sim_count = 200
    sims = sim_count
    while sims > 0:
        test_army = combat_sim(enemy_comp, test_comp)
        if (test_type == 'units') and (len(test_army) > test_limit):
            wins += 1
        elif (test_type == 'health') and (get_health(test_army) > test_limit):
            wins += 1
        elif (test_type == 'damage') and (get_damage(test_army) > test_limit):
            wins += 1
        sims -= 1
    if (wins / sim_count) > 0.9:
        return 1
    else:
        return 0

First build of our main code only deals with one linear program (optimize for resource cost). Later builds should include a parameter so that we can reuse most of the code for other optimizations such as total build time.

Right now, resource cost is calculated much in the same way the army value stat is calculated in-game; the resource cost of a unit is the sum of the gas and mineral cost of that unit. How much a player actually values gas vs minerals usually depends on how much they currently have available and how much they are producing. A skilled player might also have an idea of what sort of units or research they want to be building.

Recall for the linear program, objective function is to minimize
$$\sum_{name} cost[name] \cdot count[name]$$
with the two constraints being stay below the supply cap
$$\sum_{name} count[name] \cdot supply[name] \leq 200$$
and make sure that army comp is viable
$$viable_{army} \geq 1$$

The task at hand is to figure out how to deal with creating these variables in our linear program. The viable variable is per army composition, and so to initialize it we would need to preconstruct and itterate through all possible army comps with supply $\leq$ 200, then assign each of those comps a viable variable.

Basically, the issue is that the viable variable deals with army compositions as a whole, not individual units

In [5]:
def init_army_comps(race, supply_cap=200):
    """
    Input is the race we wish to build army comps for
    Creates a list of all possible army compositions with
    total supply <= supply_cap (default 200)
    Returns that list of army compositions
    """
    race_units = {}
    if race == 'Terran':
        race_units = list(get_Terran().keys())
    elif race == 'Protoss':
        race_units = list(get_Protoss().keys())
        # Intercceptors are a special case
        race_units.remove('Interceptor')
    elif race == 'Zerg':
        race_units = list(get_Zerg().keys())
    
    comps = []
    
    # create all possible combinations of army comps
    base = {}
    for name in race_units:
        base[name] = 0
    # note in final version, it will be range(supply_cap)
    # we use range(3) for testing since it is faster
    prod = product(range(3), repeat=len(base))
    prod = list(prod)
    for combo in prod:
        comp = base
        i = 0
        for n in base:
            comp[n] = combo[i]
            i += 1
            temp_comp = comp.copy()
            if temp_comp not in comps:
                comps.append(temp_comp)
    # remove all comps with more than one Mothership
    # or more than 200 supply
    for comp in comps:
        if (get_army_supply(comp) > supply_cap) or extra_Motherships(comp):
            comps.remove(comp)
    
    return comps

In [13]:
with open('Terran_comps.json', 'w') as fout:
    json.dump(init_army_comps('Terran'), fout)

Use this to read JSON file

In [14]:
with open(r"Terran_comps.json", "r") as read_file:
    data = json.load(read_file)
print(data)

[{'Marine': 0, 'SCV': 0, 'Medivac': 0, 'Hellion': 0}, {'Marine': 0, 'SCV': 0, 'Medivac': 0, 'Hellion': 1}, {'Marine': 0, 'SCV': 0, 'Medivac': 0, 'Hellion': 2}, {'Marine': 0, 'SCV': 0, 'Medivac': 1, 'Hellion': 2}, {'Marine': 0, 'SCV': 0, 'Medivac': 1, 'Hellion': 0}, {'Marine': 0, 'SCV': 0, 'Medivac': 1, 'Hellion': 1}, {'Marine': 0, 'SCV': 0, 'Medivac': 2, 'Hellion': 2}, {'Marine': 0, 'SCV': 0, 'Medivac': 2, 'Hellion': 0}, {'Marine': 0, 'SCV': 0, 'Medivac': 2, 'Hellion': 1}, {'Marine': 0, 'SCV': 1, 'Medivac': 2, 'Hellion': 2}, {'Marine': 0, 'SCV': 1, 'Medivac': 0, 'Hellion': 2}, {'Marine': 0, 'SCV': 1, 'Medivac': 0, 'Hellion': 0}, {'Marine': 0, 'SCV': 1, 'Medivac': 0, 'Hellion': 1}, {'Marine': 0, 'SCV': 1, 'Medivac': 1, 'Hellion': 2}, {'Marine': 0, 'SCV': 1, 'Medivac': 1, 'Hellion': 0}, {'Marine': 0, 'SCV': 1, 'Medivac': 1, 'Hellion': 1}, {'Marine': 0, 'SCV': 1, 'Medivac': 2, 'Hellion': 0}, {'Marine': 0, 'SCV': 1, 'Medivac': 2, 'Hellion': 1}, {'Marine': 0, 'SCV': 2, 'Medivac': 2, 'Hellio

In [9]:
init_army_comps('Terran')

[{'Marine': 0, 'SCV': 0, 'Medivac': 0, 'Hellion': 0},
 {'Marine': 0, 'SCV': 0, 'Medivac': 0, 'Hellion': 1},
 {'Marine': 0, 'SCV': 0, 'Medivac': 0, 'Hellion': 2},
 {'Marine': 0, 'SCV': 0, 'Medivac': 1, 'Hellion': 2},
 {'Marine': 0, 'SCV': 0, 'Medivac': 1, 'Hellion': 0},
 {'Marine': 0, 'SCV': 0, 'Medivac': 1, 'Hellion': 1},
 {'Marine': 0, 'SCV': 0, 'Medivac': 2, 'Hellion': 2},
 {'Marine': 0, 'SCV': 0, 'Medivac': 2, 'Hellion': 0},
 {'Marine': 0, 'SCV': 0, 'Medivac': 2, 'Hellion': 1},
 {'Marine': 0, 'SCV': 1, 'Medivac': 2, 'Hellion': 2},
 {'Marine': 0, 'SCV': 1, 'Medivac': 0, 'Hellion': 2},
 {'Marine': 0, 'SCV': 1, 'Medivac': 0, 'Hellion': 0},
 {'Marine': 0, 'SCV': 1, 'Medivac': 0, 'Hellion': 1},
 {'Marine': 0, 'SCV': 1, 'Medivac': 1, 'Hellion': 2},
 {'Marine': 0, 'SCV': 1, 'Medivac': 1, 'Hellion': 0},
 {'Marine': 0, 'SCV': 1, 'Medivac': 1, 'Hellion': 1},
 {'Marine': 0, 'SCV': 1, 'Medivac': 2, 'Hellion': 0},
 {'Marine': 0, 'SCV': 1, 'Medivac': 2, 'Hellion': 1},
 {'Marine': 0, 'SCV': 2, 'Me

In [6]:
def extra_Motherships(comp):
    """
    Input is an army composition
    Returns True if more than one Mothership is
    in that army, else returns False
    """
    if 'Mothership' in comp:
        if comp['Mothership'] > 1:
            return True
    else:
        return False

In [7]:
def get_army_supply(comp):
    """
    Input is an army composition
    Returns the total supply used by that army
    """
    supply_total = 0
    Units = get_Units()
    for name in comp:
        supply = Units[name]['supply'] * comp[name]
        supply_total += supply
    return supply_total

Main function for our linear program will be below

In [ ]:
def find_optimal_army(enemy_comp, test_type='cost', race='Terran', viab='units'):
    """
    Input is the army composition we are trying to optimize against
    test_type is a string, either "cost" or "time", determines what we
    are optimizing for. race is a string, either "Terran", "Protoss", or "Zerg",
    that we are creating our test army from.
    viab is a string, either 'units', 'health', or 'damage' that determines
    our viability function
    Finds an army composition of the given race that is the most efficent in terms of test_type
    Returns that optimal army composition
    """
    army_model = Model("StarCraftII Army")
    
    
    